In [1]:
import mujoco
import os
from mujoco import viewer
import re

In [2]:
# Get the absolute path to your FR3 model
current_dir = os.path.abspath(os.getcwd())
fr3_dir = os.path.join(current_dir, "franka_fr3")
fr3_xml_path = os.path.join(fr3_dir, "fr3.xml")

In [3]:
model = mujoco.MjModel.from_xml_path(fr3_xml_path)
data = mujoco.MjData(model)

In [5]:
"""
Create a single franka model with a table and a movable box.
"""

with open(fr3_xml_path, 'r') as f:
    fr3_xml_content = f.read()

# Remove all keyframe definitions
fr3_xml_content = re.sub(r'<keyframe.*?</keyframe>', '', fr3_xml_content, flags=re.DOTALL)

# Modify the XML to add table and box
if '</worldbody>' in fr3_xml_content:
    modified_xml = fr3_xml_content.replace('</worldbody>', '''
    <!-- Table -->
    <body name="table" pos="0.5 0.5 -0.05">
        <geom name="tabletop" type="box" size="0.7 0.7 0.05" rgba="0.8 0.8 0.8 1" mass="10"/>
        <geom name="leg1" type="box" pos="0.55 0.55 -0.4" size="0.025 0.025 0.4" rgba="0.8 0.8 0.8 1" mass="1"/>
        <geom name="leg2" type="box" pos="0.55 -0.55 -0.4" size="0.025 0.025 0.4" rgba="0.8 0.8 0.8 1" mass="1"/>
        <geom name="leg3" type="box" pos="-0.55 0.55 -0.4" size="0.025 0.025 0.4" rgba="0.8 0.8 0.8 1" mass="1"/>
        <geom name="leg4" type="box" pos="-0.55 -0.55 -0.4" size="0.025 0.025 0.4" rgba="0.8 0.8 0.8 1" mass="1"/>
    </body>
    
    <!-- Movable Box - with freejoint for full 6DOF movement (including XYZ translation) -->
    <body name="box" pos="0.7 0 0.1">
        <freejoint name="box_freejoint"/>
        <geom name="box_geom" type="box" size="0.05 0.05 0.05" rgba="0.5 1 0.8 1"/>
        <inertial pos="0 0 0" mass="3" diaginertia="0.001 0.001 0.001"/>
    </body>
</worldbody>''')

    save_file_path = os.path.join(fr3_dir, "fr3_with_moveable_box.xml")
    
    # Save the modified XML permanently
    with open(save_file_path, 'w') as f:
        f.write(modified_xml)
    
    # Test load the saved model
    model = mujoco.MjModel.from_xml_path(save_file_path)
    data = mujoco.MjData(model)
    print("Saved model loads successfully!")

Saved model loads successfully!


In [5]:
"""
Create a dual FR3 scene by copying the original FR3 model and translating the second robot.
"""

def dual_fr3(fr3_xml_path, save_dir):
    """
    copy the original FR3 XML, rename elements for two robots, and translate the second robot.
    """
    
    # Read the original FR3 XML file
    with open(fr3_xml_path, 'r') as f:
        original_xml = f.read()
    
    # Remove keyframes to avoid conflicts
    # Keyframes in Mujoco define predefined positions for joints
    # Like "home position" or "initial position"
    # when duplicating robots, we need to remove these to avoid conflicts
    xml_content = re.sub(r'<keyframe.*?</keyframe>', '', original_xml, flags=re.DOTALL)
    
    # Find the worldbody section
    worldbody_pattern = r'(<worldbody[^>]*>)(.*?)(</worldbody>)'
    worldbody_match = re.search(worldbody_pattern, xml_content, re.DOTALL)
    
    worldbody_start = worldbody_match.group(1)  # <worldbody>
    worldbody_content = worldbody_match.group(2)  # content inside
    worldbody_end = worldbody_match.group(3)    # </worldbody>
    
    # Rename first robot elements
    first_robot_content = worldbody_content
    first_robot_content = re.sub(r'name="([^"]*)"', r'name="\1_robot1"', first_robot_content)
    first_robot_content = re.sub(r'joint="([^"]*)"', r'joint="\1_robot1"', first_robot_content)
    
    # Create second robot by copying and renaming
    second_robot_content = worldbody_content
    
    # Rename all elements for second robot
    second_robot_content = re.sub(r'name="([^"]*)"', r'name="\1_robot2"', second_robot_content)
    second_robot_content = re.sub(r'joint="([^"]*)"', r'joint="\1_robot2"', second_robot_content)
    
    # Wrap second robot in a translated body
    translated_robot = f'''
    <!-- Second FR3 Robot (translated) -->
    <body name="robot2_base" pos="0 1 0">
{second_robot_content}
    </body>'''
    
    # Create the new worldbody with both robots
    new_worldbody_content = f'''{worldbody_start}  
    <!-- First FR3 Robot (renamed) -->
{first_robot_content}
{translated_robot}
    
    <!-- Table-->
    <body name="table" pos="0.5 0.5 -0.05">
        <geom name="table_surface" type="box" size="1 1 0.05" rgba="0.7 0.7 0.7 1" mass="3"/>
    </body>
    
    <!-- Movable object -->
    <body name="box" pos="0.5 0 0.1">
        <freejoint name="object_joint"/>
        <geom name="object_geom" type="box" size="0.05 0.05 0.05" rgba="0.5 0.2 0.2 1"/>
        <inertial pos="0 0 0" mass="1" diaginertia="0.0001 0.0001 0.0001"/>
    </body>
    
    <!-- Movable object -->
    <body name="box_2" pos="0.5 1 0.1">
        <freejoint name="object_joint_2"/>
        <geom name="object_geom_2" type="box" size="0.05 0.05 0.05" rgba="0.5 0.5 0.2 1"/>
        <inertial pos="0 0 0" mass="1" diaginertia="0.0001 0.0001 0.0001"/>
    </body>
    
{worldbody_end}'''
    
    # Handle actuators - copy and rename for second robot
    actuator_pattern = r'(<actuator[^>]*>)(.*?)(</actuator>)'
    actuator_match = re.search(actuator_pattern, xml_content, re.DOTALL)
    
    if actuator_match:
        actuator_start = actuator_match.group(1)
        actuator_content = actuator_match.group(2)
        actuator_end = actuator_match.group(3)
        
        # Create renamed actuators for first robot
        first_actuators = re.sub(r'joint="([^"]*)"', r'joint="\1_robot1"', actuator_content)
        first_actuators = re.sub(r'name="([^"]*)"', r'name="\1_robot1"', first_actuators)
        
        # Create second set of actuators
        second_actuators = re.sub(r'joint="([^"]*)"', r'joint="\1_robot2"', actuator_content)
        second_actuators = re.sub(r'name="([^"]*)"', r'name="\1_robot2"', second_actuators)
        
        new_actuator_section = f'''{actuator_start}
    <!-- First robot actuators -->
{first_actuators}
    
    <!-- Second robot actuators -->
{second_actuators}
{actuator_end}'''
        
        # Replace actuator section in xml
        xml_content = re.sub(actuator_pattern, new_actuator_section, xml_content, flags=re.DOTALL)
    
    # Replace worldbody in the complete XML
    final_xml = re.sub(worldbody_pattern, new_worldbody_content, xml_content, flags=re.DOTALL)
    
    # Update model name
    final_xml = re.sub(r'model="[^"]*"', 'model="dual_fr3_scene"', final_xml)
    
    # Save the result
    save_path = os.path.join(save_dir, "dual_fr3.xml")
    
    with open(save_path, 'w') as f:
        f.write(final_xml)
    
    return save_path, model, data

# Main execution
if __name__ == "__main__":
    
    print("Creating dual FR3 scene using copy and translate approach...")
    save_path, model, data = dual_fr3(fr3_xml_path, fr3_dir)
    
    ## Test scene loading
    model = mujoco.MjModel.from_xml_path(save_path)
    data = mujoco.MjData(model)
    print("Model created and saved successfully")

Creating dual FR3 scene using copy and translate approach...
Model created and saved successfully


In [ ]:
# Create a viewer and visualize the model
viewer.launch(model, data)

/home/kai/.local/lib/python3.10/site-packages/glfw/__init__.py:917: GLFWError: (65548) b'Wayland: The platform does not provide the window position'
  warnings.warn(message, GLFWError)


: 